
# Quantile regression workflows

<span style="font-size: 16pt; font-style: italic; font-weight: bold">PyData Global 2024</span>

December 2024

----

## Abstract

### Short

This talk showcases and exemplifies the rapid specification and execution of Quantile Regression workflows. Various use cases are discussed, including fitting, outlier detection, conditional CDFs, and simulations, using different types of time series data.

### Longer


Quantile Regression (QR) is a powerful analysis method that is often considered superior to other regression techniques. The benefits of QR are highlighted in this talk through multiple examples from various "real-life" time series, such as finance, weather, and human activities. The use of QR for fitting, outlier detection, conditional CDFs, and simulations will be demonstrated. The analysis is significantly simplified with the new Python package ["Regressionizer"](https://pypi.org/project/Regressionizer/), [AAp1], which allows for quick setup and execution of QR workflows.

This presentation is intended for data analysts, data scientists, engineers, and anyone with an interest in time series analysis. A basic understanding of Python is all that is required from the audience, as the coding pipelines have been designed to be straightforward and easy to follow.

### Teaser

![](https://raw.githubusercontent.com/antononcube/MathematicaVsR/refs/heads/master/Projects/QuantileRegressionWorkflows/Presentation-documents-useR-ODSC-Boston-2019-04-18/0-XKCD-2048-vs-QRMon.png)


------

## Workflows flowchart

The following flowchart summarizes the workflows that are supported by [`Regressionizer`](https://pypi.org/project/Regressionizer/):

![](https://raw.githubusercontent.com/antononcube/Python-Regressionizer/main/docs/img/Quantile-regression-workflow-extended.jpg)

-----

##  History





- Quantile Regression (QR) started mid 17th century
    - For Astronomy-related problems 
        - Similar to Linear Regression (LR)
- Recently (≈ 50 years ago) Roger Koenker et al. introduced modern computational framework for QR
    - And worked out QR-based inference

### Previous work on software packages

Roger Koenker implemented the R package "quantreg", [RKp1].
Anton Antonov implemented the R package "QRMon-R" for the specification of monadic pipelines for doing QR, [AAp1].

Several Wolfram Language (aka Mathematica) packages are implemented by Anton Antonov, see [AAp1, AAp2, AAf1].

**Remark:** The paclets at the Wolfram Language Paclet Repository were initially Mathematica packages hosted at GitHub.
The Wolfram Function Repository function
[`QuantileRegression`](https://resources.wolframcloud.com/FunctionRepository/resources/QuantileRegression/), [AAf1]
does only B-spline fitting.


------

## Summary of `Regressionizer` features


- The primary focus of `Regressionizer` is [Quantile Regression (QR)](https://en.wikipedia.org/wiki/Quantile_regression), [RK1, RK2].

- It closely follows the monadic pipeline design explained in detail in the document
["A monad for Quantile Regression workflows"](https://github.com/antononcube/MathematicaForPrediction/blob/master/MarkdownDocuments/A-monad-for-Quantile-Regression-workflows.md), [AA1].

- The class `Regressionizer` facilitates rapid specifications of regressions workflows.
  - To quickly specify:
    - data rescaling and summary
    - regression computations
    - outliers finding
    - conditional Cumulative Distribution Functions (CDFs) reconstruction
    - plotting of data, fits, residual errors, outliers, CDFs

- `Regressionizer` works with data frames, numpy arrays, lists of numbers, and lists of numeric pairs.

### Proof (via LLM utilization)

In [ ]:
%%bash
dsl-translation --to-language=Python "DSL MODULE QRMon; create from dfFinance;
compute quantile regression with 20 knots and interpolation order 2;
show error plots"

------

## Definitions and details

- The curves computed with Quantile Regression are called **regression quantiles**.

- `Regressionizer` has three regression methods:
  - `quantile_regression`
  - `quantile_regression_fit`
  - `least_squares_fit`

- The regression quantiles computed with the methods `quantile_regression` and `quantile_regression_fit`
  correspond to probabilities specified with the argument `probs`.

- The method`quantile_regression` computes fits using a B-spline functions basis.
  - The basis is specified with the arguments `knots` and `order`.
  - `order` is 3 by default.

- The methods `quantile_regession_fit` and `least_squares_fit` use lists of basis functions to fit with
  specified with the argument `funcs`.

-------

## Setup

Load the "Regressionizer" and other "standard" packages:

In [ ]:
from Regressionizer import *
from OutlierIdentifiers import *

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp

In [ ]:
template='plotly_dark'
data_color='darkgray'

### Temperature data

In [ ]:
url = "https://raw.githubusercontent.com/antononcube/MathematicaVsR/master/Data/MathematicaVsR-Data-Atlanta-GA-USA-Temperature.csv"
dfTemperature = pd.read_csv(url)
dfTemperature['DateObject'] = pd.to_datetime(dfTemperature['Date'], format='%Y-%m-%d')
dfTemperature = dfTemperature[(dfTemperature['DateObject'].dt.year >= 2020) & (dfTemperature['DateObject'].dt.year <= 2023)]
dfTemperature

### Financial data

In [ ]:
dirName = "../../../Python-Regressionizer/Regressionizer/resources"
fileName = dirName + "/dfFinancialData.csv.zip"
dfFinancialData = pd.read_csv(fileName, compression='zip')
dfFinancialData['Time'] = pd.to_datetime(dfFinancialData['Time'], format='%Y-%m-%d')
dfFinancialData['Time'] = dfFinancialData['Time'].apply(lambda x: x.timestamp())

dfFinancialData


### Distribution data

Generate random data:

In [ ]:
np.random.seed(0)
x = np.linspace(0, 2, 300)
y = np.sin(2 * np.pi * x) + np.random.normal(0, 0.4, x.shape)
data = np.column_stack((x, y))

In [ ]:
xs = np.arange(-3, 3.01, 0.01)
dfDistributionData = pd.DataFrame({
    'X': xs,
    'Y': [np.exp(-x**2) + np.random.normal(0, 0.15 * np.sqrt(abs(1.5 - x) / 1.5)) for x in xs]
})
dfDistributionData

In [ ]:
data = dfDistributionData.to_numpy()

Plot the generated data:

In [ ]:
fig = px.scatter(x=data[:, 0], y=data[:, 1], labels={'x': 'X-axis', 'y': 'Y-axis'}, template=template, width = 800, height = 600)
fig.show()

------

## Fit given functions

Define a list of functions:

In [ ]:
funcs = [lambda x: 1, lambda x: x, lambda x: np.cos(x), lambda x: np.cos(3 * x), lambda x: np.cos(6 * x)]

In [ ]:
def chebyshev_t_polynomials(n):
    if n == 0:
        return lambda x: 1
    elif n == 1:
        return lambda x: x
    else:
        T0 = lambda x: 1
        T1 = lambda x: x
        for i in range(2, n + 1):
            Tn = lambda x, T0=T0, T1=T1: 2 * x * T1(x) - T0(x)
            T0, T1 = T1, Tn
        return Tn

chebyshev_polynomials = [chebyshev_t_polynomials(i) for i in range(8)]

Define ***regression quantile*** probabilities:

In [ ]:
probs = [0.1, 0.5, 0.9]

Perform Quantile Regression and (non-linear) Least Squares Fit:

In [ ]:
obj2 = (
    Regressionizer(data)
    .echo_data_summary()
    .quantile_regression_fit(funcs=chebyshev_polynomials, probs=probs)
    .least_squares_fit(funcs=chebyshev_polynomials)
    .plot(title = "Quantile Regression and Least Squares fitting using Chebyshev polynomials", template=template)
)

Plot the obtained regression quantilies and least squares fit:

In [ ]:
obj2.take_value().show()

The ***regression quantiles** separate the data according to the given probabilities:

In [ ]:
obj2.separate(cumulative=True, fractions=True).take_value()

-------

## Fit B-splines

Instead of coming-up with basis functions we can use B-spline basis:

In [ ]:
obj = Regressionizer(data).quantile_regression(knots=8, probs=[0.2, 0.5, 0.8]).plot(title="B-splines fit", template=template)

Show the obtained plot:

In [ ]:
obj.take_value().show()

Here is a dictionary of the found regression quantiles:

In [ ]:
obj.take_regression_quantiles()

------

## Weather temperature data

Convert to "numpy" array: 

In [ ]:
temp_data = dfTemperature[['AbsoluteTime', 'Temperature']].to_numpy()
temp_data.shape

Here is pipeline for Quantile Regression computation and making of a corresponding plot:

In [ ]:
obj = (
    Regressionizer(temp_data)
    .echo_data_summary()
    .quantile_regression(knots=20, probs=[0.2, 0.5, 0.8])
    .date_list_plot(title="Atlanta, Georgia, USA, Temperature, ℃", template=template, data_color=data_color, width = 1200)
)

Show the obtained plot:

In [ ]:
obj.take_value().show()

Here we show the fractions of the number of points under each regression quantile:

In [ ]:
obj.separate(cumulative=True, fractions=True).take_value()

**Remark:** If the quantile regression algorithms work correctly then the cumulation separation fractions correspond -- i.e. are nearly equal - to the probabilities of the regression quantiles.

-------

## Fitting errors

### Errors

Here the absolute fitting errors are computed and the average is for each is computed:

In [ ]:
{ k : np.mean(np.array(d)[:,1]) for k, d in obj.errors(relative_errors=False).take_value().items() }

### Error plots

Here we give the fitting errors (residuals) for the regression quantiles found and plotted above:

In [ ]:
obj.error_plots(relative_errors=False, date_plot=True, template=template, width=1200, height=300).take_value().show()

------

## Outliers

One way to find _contextual_ outliers in time series is to find regression quantiles at low- and high enough probabilities, and then select the points "outside" of those curves:

In [ ]:
obj = (
    Regressionizer(temp_data)
    .quantile_regression(knots=20, probs=[0.01,  0.99], order=3)
    .outliers()
)

obj.take_value()

Here we plot the outliers (using a "narrower band" than above):

In [ ]:
obj = (
    Regressionizer(temp_data)
    .quantile_regression(knots=20, probs=[0.05,  0.95], order=3)
    .outliers_plot(
        title="Outliers of Atlanta, Georgia, USA, Temperature, ℃",
        date_plot=True, 
        template=template,
        data_color=data_color,
        width = 1200, height = 400)
)

obj.take_value().show()

------

## Point anomalies

Here is pipeline for Quantile Regression computation and making of a corresponding plot:

In [ ]:
obj = (
    Regressionizer(dfFinancialData.to_numpy())
    .echo_data_summary()
    .quantile_regression(knots=20, probs=[0.2])
    .date_list_plot(title="Financial data", template=template, data_color=data_color, width = 1200)
)

Show the obtained plot:

In [ ]:
fig = obj.take_value()
fig.show()


In [ ]:
outliers = (obj
.find_anomalies_by_residuals(
    relative_errors=True,
    threshold=None, 
    outlier_identifier=quartile_identifier_parameters)
.take_value());

fig.add_trace(go.Scatter(x=to_datetime_index(outliers[:,0]), y=outliers[:,1], mode='markers', name='Outliers', marker_color = "orange"))

In [ ]:
outliers = (obj
.find_anomalies_by_residuals(
    relative_errors=False,
    threshold=None, 
    outlier_identifier=hampel_identifier_parameters)
.take_value());

fig.add_trace(go.Scatter(x=to_datetime_index(outliers[:,0]), y=outliers[:,1], mode='markers', name="Outliers 2", marker_color = "Red"))

---------

## Pick points along regression quantiles  

Here is a workflow that finds a regression quantile for probability `0.5` and pick data points around it:

In [ ]:
probs = [0.5,]
obj=(
    Regressionizer(temp_data)
    .quantile_regression(knots=20, probs=probs)
    .pick_path_points(threshold=1.5, relative_errors=False)
    )

path_points = np.array(obj.take_value()[probs[0]])
dfPathPoints = pd.DataFrame(path_points)
dfPathPoints

Get the a `Figure` object from the `Regressionizer` object:

In [ ]:
fig = obj.date_plot(width=1200, template=template, data_color=data_color).take_value()

Plot the data and the path points:

In [ ]:
fig.add_trace(go.Scatter(x=to_datetime_index(path_points[:,0]), y=path_points[:,1], mode='markers', name='Picked points'))
fig.show()

Plot just the path points:

In [ ]:
fig = px.scatter(x=dfPathPoints.iloc[:,0], y=dfPathPoints.iloc[:,1], template=template, width=1200)

fig.show()

--------

## Conditional CDF

Here is a list of probabilities to be used to reconstruct Cumulative Distribution Functions (CDFs):

In [ ]:
probs = np.sort(np.concatenate((np.arange(0.1, 1.0, 0.1), [0.01, 0.99])))
probs

Here we find the regression quantiles for those probabilities:

In [ ]:
obj=(
    Regressionizer(temp_data)
    .quantile_regression(knots=20,probs=probs)
    .date_list_plot(template=template, data_color="darkgray", width=1200)
    )

Here we show the plot obtained above:

In [ ]:
obj.take_value().show()

### Get CDF function

Here we take a date in ISO format and convert to number of seconds since 1900-01-01:

In [ ]:
from datetime import datetime

iso_date = "2022-01-01"
date_object = datetime.fromisoformat(iso_date)
epoch = datetime(1900, 1, 1)

focusPoint = int((date_object - epoch).total_seconds())
print(focusPoint)

Here the _conditional_ CDF at that date is computed:

In [ ]:
aCDFs = obj.conditional_cdf(focusPoint).take_value()
aCDFs

Plot the obtained CDF function:

In [ ]:
xs = np.linspace(obj.take_regression_quantiles()[0.01](focusPoint), obj.take_regression_quantiles()[0.99](focusPoint), 20)
cdf_values = [aCDFs[focusPoint](x) for x in xs]

fig = go.Figure(data=[go.Scatter(x=xs, y=cdf_values, mode='lines')])
# Update layout
fig.update_layout(
    title='Temperature Data CDF at ' + str(focusPoint),
    xaxis_title='Temperature',
    yaxis_title='Probability',
    template=template,
    legend=dict(title='Legend'),
    height=300,
    width=800
)
fig.show()

### Plot multiple CDFs

Here are few dates converted into number of seconds since 1990-01-01:

In [ ]:
pointsForCDFs = [focusPoint + i * 365 * 24 * 3600 for i in range(-1,2)]
pointsForCDFs

Here are the plots of CDF at those dates:

In [ ]:
obj.conditional_cdf_plot(pointsForCDFs, title = 'CDFs', template=template).take_value().show()

-----

## Simulation

Here is a pipeline that produces simulated time series based on fitted regression quantiles:

In [ ]:
sim_points = (
    Regressionizer(temp_data)
    .echo_data_summary()
    .quantile_regression(knots=20, probs=[0.001, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.999])
    .simulate(800)
    .take_value()
)

Here is the plot of the original and simulated time series:

In [ ]:
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=['Original time series', 'Simulated time series'])
fig.add_trace(go.Scatter(x=to_datetime_index(temp_data[:,0]), y=temp_data[:,1], mode='lines', name=None), row = 1, col = 1)
fig.add_trace(go.Scatter(x=to_datetime_index(sim_points[:,0]), y=sim_points[:,1], mode='lines', name=None), row = 2, col = 1)
fig.update_layout(template=template, title = "Weather temperature data", showlegend=False)
fig.show()

-----

## Big picture

- Having a "complete" set of Machine Learning (ML) workflows in several programming languages.
  - The focus is on Python, R, and Wolfram Language (WL).
- Python's Quantile Regression pipeline implementation was missing for years.
- Code generation using both:
  - Large Language Models (LLM)
  - Grammar-based interpreters (ie. "Small Language Models")

----- 

## LLM support 

Code generation via LLMs can be done using:

1. Examples of natural language phrases/commands to pipeline segments
2. Question Answering System (QAS)

Here is an example of QSA via LLMs:

In [ ]:
%%bash
concretize -l=Python Make a quantile regression pipeline, using 20 notes and the data set dfTemperature

In [ ]:
qrObj = (Regressionizer(dfTemperature[["AbsoluteTime", "Temperature"]].to_numpy())
.echo_data_summary()
.quantile_regression(knots = 20, probs = [0.25, 0.5, 0.75], order = 3)
.plot(date_plot = True, template=template)
)

In [ ]:
qrObj.take_value()

-----

## Conclusion 

### Questions

### Anticipated questions


- Can we do Multi-dimensional Quantile Regression (QR)?
- What happens when fitting through only few points?
- What other anomaly detection methods can be used?
- Is there are a list of all "good" features QR?

------

## References

### Articles, books

[RK1] Roger Koenker, 
[Quantile Regression](https://books.google.com/books/about/Quantile_Regression.html?id=hdkt7V4NXsgC), 
Cambridge University Press, 2005.

[RK2] Roger Koenker,
["Quantile Regression in R: a vignette"](https://cran.r-project.org/web/packages/quantreg/vignettes/rq.pdf),
(2006),
[CRAN](https://cran.r-project.org/).

[AA1] Anton Antonov,
["A monad for Quantile Regression workflows"](https://github.com/antononcube/MathematicaForPrediction/blob/master/MarkdownDocuments/A-monad-for-Quantile-Regression-workflows.md),
(2018),
[MathematicaForPrediction at GitHub](https://github.com/antononcube/MathematicaForPrediction).

### Packages, paclets

[AAp1] Anton Antonov,
[Quantile Regression Python package](https://github.com/antononcube/Python-Regressionizer),
(2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[`QRMon-R`](https://github.com/antononcube/QRMon-R),
(2019),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[Quantile Regression WL paclet](https://github.com/antononcube/WL-QuantileRegression-paclet),
(2014-2023),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[Monadic Quantile Regression WL paclet](https://github.com/antononcube/WL-MonadicQuantileRegression-paclet),
(2018-2024),
[GitHub/antononcube](https://github.com/antononcube).

[AAf1] Anton Antonov,
[`QuantileRegression`](https://resources.wolframcloud.com/FunctionRepository/resources/QuantileRegression),
(2019),
[Wolfram Function Repository](https://resources.wolframcloud.com/FunctionRepository/resources/QuantileRegression).

[RKp1] Roger Koenker,
[`quantreg`](https://cran.r-project.org/web/packages/quantreg/index.html),
[CRAN](https://cran.r-project.org/).

### Repositories

[AAr1] Anton Antonov,
[DSL::English::QuantileRegressionWorkflows in Raku](https://github.com/antononcube/Raku-DSL-English-QuantileRegressionWorkflows),
(2020),
[GitHub/antononcube](https://github.com/antononcube/Raku-DSL-English-QuantileRegressionWorkflows).


### Videos

[AAv1] Anton Antonov,
["Boston useR! QuantileRegression Workflows 2019-04-18"](https://www.youtube.com/watch?v=a_Dk25xarvE),
(2019),
[Anton Antonov at YouTube](https://www.youtube.com/@AAA4Prediction).

[AAv2] Anton Antonov,
["useR! 2020: How to simplify Machine Learning workflows specifications"](https://www.youtube.com/watch?v=b9Uu7gRF5KY),
(2020),
[R Consortium at YouTube](https://www.youtube.com/channel/UC_R5smHVXRYGhZYDJsnXTwg).